In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

[[Home](/)]
[[Webapps](/jupyter/voila/tree/webapps)]

# People at vk.com



In [2]:
import pandas as pd
from stratosphere import Stratosphere
from stratosphere.utils.inspect_flows import InspectFlows
from stratosphere import options
from stratosphere.services.extractor import Extractor
from stratosphere.storage.models import Flow
import json
from IPython.display import Image, HTML



In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

In [4]:
kb = Stratosphere(options.get("db.url_kb"))


In [9]:
q = """
SELECT
    *
FROM entities
where type is 'vk.com/person'
order by length(data) desc
"""

def get_df_people():
    df_people = kb.db.pandas(q)
    df_people_data = pd.json_normalize(df_people.data.apply(lambda t: json.loads(t)).values)
    df_people_data['id'] = df_people['id']
    df_people = df_people.merge(df_people_data, on='id')
    return df_people

#get_df_people().head()


In [10]:
import pandas as pd
from IPython.display import Image, HTML
import pandas as pd
import textwrap
from datetime import datetime
#from ipywidgets import HTML
from ipywidgets import GridspecLayout, Layout, Button, Textarea
import ipywidgets as widgets

import ipywidgets as widgets

def render_img_url(url):
    if not url:
        return ""
    return '<img src="'+ url + '" style=max-height:124px;"/>'

def render_href(href_col):    
    if not href_col[1]:
        return ""
    return f'<a target="_blank" href={href_col[1]}>{href_col[0]}</a>'


def render_df(df):
    if len(df) == 0:
        print('No matches.')
        return
    
    print(f"Found {len(df)} matching profiles (limit 100)")
    print()
    
    df = df.head(100)

    df = df.copy()
    df['user_id'] = df.user_ids.apply(lambda x: json.loads(x)[0])

   
    df['bdate'] = df.bdate.fillna("").apply(lambda x: f'"{x}"' if x else "")
    df['user_page'] = df.apply(lambda row: [row.full_name, f'https://vk.com/id{row.user_id}'], axis=1)
    df['friends_page'] = df.apply(lambda row: ['friends', f'https://vk.com/friends?id={row.user_id}&section=all'], axis=1)
    df['entity'] = df.apply(lambda row: ['e', f'/jupyter/voila/render/webapps/entity-overview.ipynb?entity_id={row.id}'], axis=1)
    cols = ['entity', 'user_page', 'friends_page', 'country', 'city', 'bdate', 'skype', 'url_picture']
    df = df[cols]
    df = df.fillna("")
    
    display(HTML(df.to_html(escape=False, formatters=dict(entity=render_href, friends_page=render_href, user_page=render_href, url_picture=render_img_url)))) # escape=False ,formatters=dict(url_picture=render_img_url))
    
    #display_network(df)
#render_df(df_people.head(5))

In [25]:
from pyvis.network import Network
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, IFrame

    
def display_network(df):
    entity_ids = ','.join(f'"{entity_id}"' for entity_id in df.id.values)

    def get_ents_rels(min_conn=2):
    
        while True:
            q = f"""
with entities as (
select src as id from relationships where type = 'friend'
union all
select dst as id from relationships where type = 'friend'
),

target_entities as (
select id
from entities
group by 1
having count(*) >={min_conn})

select * from relationships 
    where dst in (select id from target_entities) 
    and  src in (select id from target_entities)

    """

            df_rels = kb.db.pandas(q)
    
    
            q = f"""
with entities as (
select src as id from relationships where type = 'friend'
union all
select dst as id from relationships where type = 'friend'
),

target_entities as (
select id
from entities
group by 1
having count(*) >={min_conn})

select * from target_entities

    """
    
            df_ents = kb.db.pandas(q)
        
            if len(df_rels) < 100:
                return df_rels, df_ents
            
            min_conn+=1
            
    
    df_rels, df_ents  = get_ents_rels()
    
#    display(df_rels)       
#    nodes = list(set(df_rels.src.tolist()).union(set(df_rels.dst.tolist())))
    nodes = df_ents.id.tolist()
    
    net = Network(notebook=True, height='450px', cdn_resources='in_line', neighborhood_highlight=True, directed=False) #, select_menu=True, filter_menu=True)
#    net.show_buttons(filter_='physics')
#    net.show_buttons()    
    net.set_options('{"layout": {"randomSeed":5}}')
    for idx, row in df.iterrows():
        if row.id in nodes:
            net.add_node(row.id, label=row.full_name.replace(' ', "\n").replace('-', "\n"))
    
    for idx, row in df_rels.iterrows():
        net.add_edge(row.src, row.dst)
        #print(row.src, row.dst)
        
    net.show('network-overview.html', notebook=False)
     
    display(IFrame(src='/jupyter/files/webapps/network-overview.html',width='100%', height='500px'))    
    
#display_network(get_df_people())

In [26]:


w_textsearch = widgets.Text(
    value='',
    placeholder='',
    description='Filter:',
    style = {'description_width': 'initial'},
    layout=Layout(width="99%"),
    disabled=False
)



#w_textsearch.value = "asd"

In [27]:
w_out_list = widgets.Output()
w_out_graph = widgets.Output()

In [28]:
def w_textsearch_onchange(change):
    
    mks = []
    
    if change['name'] == 'value':
        w_out_list.clear_output()
        df_people = get_df_people()
        df = df_people[df_people.data.str.lower().str.contains(w_textsearch.value.lower())]
        with w_out_list:            
            render_df(df)


In [29]:
w_textsearch.observe(w_textsearch_onchange)
w_textsearch_onchange({'name':'value'})

In [30]:
with w_out_graph:
    display_network(get_df_people())

In [31]:
# row, column

g1 = GridspecLayout(1, 1,  height='500px')
g1[0,:] = w_out_list
g2 = GridspecLayout(1, 2,  height='550px')
g2[:,0] = g1
g2[:,1] = w_out_graph

display(w_textsearch)
display(g2)

Text(value='', description='Filter:', layout=Layout(width='99%'), placeholder='', style=TextStyle(description_…

GridspecLayout(children=(GridspecLayout(children=(Output(layout=Layout(grid_area='widget001'), outputs=({'name…